In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from numpy.linalg import eigh, inv
import statsmodels.tsa.stattools as ts
from sklearn.linear_model import LinearRegression

/Users/qichaohuang/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


### 1. Perform PCA decomposition using Sample1

In [2]:
# Function to perform PCA via eigenvalue decomposition algorithm
def PCA(df, components = None):
    # Calculate the covariance matrix
    df_cov = df.cov()
    # Calculate eigenvalue and eigenvector
    Eval, Evec = eigh(df_cov)
    order = np.argsort(Eval)[::-1][:components]
    Eval, Evec = Eval[order], Evec[:, order]
    # Calculate transition
    final = np.dot(Evec.T, df.T).T
    return final, Eval, Evec

# Upload data in 'CMT-all.xlsx' into pandas data frame
di = '/users/Edith/Python/Data/'
data = pd.read_excel(di+"CMT-all.xlsx", 
                   index_col = "Date",
                   usecols = ["Date","3M", "2Y","5Y","7Y","10Y","30Y"],
                   na_values=['nan'])

# Sample
Sample1 = data.loc['20130101':'20141231']
Sample2 = data.loc['20150101':'20161231']

# Set the mean to zero
Sample1_adjust = Sample1 - Sample1.mean()
Sample2_adjust = Sample2 - Sample1.mean()

# Perform PCA decomposition using Sample1_adjust
PCA_S1 = PCA(Sample1_adjust, 2)
C = PCA_S1[2]
print('The first two factors are: \n', C, '\n')

# Pick the 2Y, 5Y, 10Y data
S = C[[1, 2, 4], :].T

A = np.mat(np.vstack([S, [0, 1, 0]]))
b = np.array([0, 0, -1]) 
x = np.linalg.solve(A,b) 
print('The (w1,w2) for butterfly porfolio weights computed by PCA are: \n',(x[0], x[2]))

IOError: [Errno 2] No such file or directory: '/users/Edith/Python/Data/CMT-all.xlsx'

To decompose the data by PCA, instead of using package, I define the function by myself. First, adjust the original data to zero mean, then calculate the eigenvalue and eigenvector of covariance matrix, the factors are computed by mutipilying the eigenvector and adjusted data.
To make the weighted FLY PCA1,2 neutral, first choose the corresponding value in factors and then make the linear combination of partial derivatives to value with respect of factor 1 and 2 to zero, then we can get the weights.

### 2. Compute Augmented Dickey-Fuller (ADF) test statistic for:  
### (1) FLY in Sample1 , (2) PCA FLY in Sample1 , (3) FLY in Sample2 , (4) PCA FLY in Sample2 

In [4]:
# Function for ADF test
def ADF_test(data, lag=0):
    adf = ts.adfuller(data, maxlag=lag)
    print('adf: ', adf[0] )
    print('p-value: ', adf[1])
    print('critical values: ', adf[4])    
    if adf[0]> adf[4]['5%']: 
        print('Time series is nonstationary.', '\n')
        # test for 1-order difference
        print('Start testing 1-order difference series...')
        temp = np.diff(Sample1_FLY)
        adf_d = ts.adfuller(temp, maxlag=0)
        print('adf: ', adf_d[0] )
        print('p-value: ', adf_d[1])
        print('critical values: ', adf_d[4])
        if adf_d[0] > adf_d[4]['5%']: 
            print('1-order difference series is nonstationary.''\n')
            # test for 2-order difference
            print('Start testing 1-order difference series...')
            temp = np.diff(Sample1_FLY, n=2)
            adf_d2 = ts.adfuller(temp, maxlag=lag)
            print('adf: ', adf_d2[0] )
            print('p-value: ', adf_d2[1])
            print('Critical values: ', adf_d2[4])
            if adf_d2[0] > adf_d2[4]['5%']: 
                print('2-order difference series is nonstationary.')
            else:
                print('2-order difference series is stationary.')
        else:
            print('1-order difference series is stationary.')
    else:
        print('Time series is stationary.')
        
# Prepare data to be tested
Sample1_2Y5Y10Y = Sample1[["2Y","5Y","10Y"]]
Sample1_FLY = np.dot(Sample1_2Y5Y10Y, [1, -1, 1])
Sample1_PCA_FLY = np.dot(Sample1_2Y5Y10Y, x)
Sample2_2Y5Y10Y = Sample2[["2Y","5Y","10Y"]]
Sample2_FLY = np.dot(Sample2_2Y5Y10Y, [1, -1, 1])
Sample2_PCA_FLY = np.dot(Sample1_2Y5Y10Y, x)

# Test
print('1. ADF test for FLY in Sample1:')
ADF_test(Sample1_FLY)
print('\n\n2. ADF test for PCA FLY in Sample1:')
ADF_test(Sample1_PCA_FLY)
print('\n\n3. ADF test for FLY in Sample2:')
ADF_test(Sample2_FLY)
print('\n\n4. ADF test for PCA FLY in Sample2:')
ADF_test(Sample2_PCA_FLY)

1. ADF test for FLY in Sample1:
('adf: ', -1.4725249211623546)
('p-value: ', 0.54705645747378495)
('critical values: ', {'5%': -2.8673495105661462, '1%': -3.4435228622952065, '10%': -2.569864247011056})
('Time series is nonstationary.', '\n')
Start testing 1-order difference series...
('adf: ', -24.983095031542941)
('p-value: ', 0.0)
('critical values: ', {'5%': -2.8673612117611267, '1%': -3.4435494520411605, '10%': -2.5698704830567247})
1-order difference series is stationary.


2. ADF test for PCA FLY in Sample1:
('adf: ', -4.976653051626335)
('p-value: ', 2.4679546380046071e-05)
('critical values: ', {'5%': -2.8673495105661462, '1%': -3.4435228622952065, '10%': -2.569864247011056})
Time series is stationary.


3. ADF test for FLY in Sample2:
('adf: ', -1.7680735912330607)
('p-value: ', 0.39642345962310011)
('critical values: ', {'5%': -2.8673378563200003, '1%': -3.4434963794639999, '10%': -2.5698580359999998})
('Time series is nonstationary.', '\n')
Start testing 1-order difference 

I define a function to perform stationary test by ADF. If the original data is not stationary, then test for the 1-order difference data, and then 2-order difference data. 
From the results, we can see that under 95% confidence level, the FLY in Sample1 is 1-order stationary, PCA FLY in Sample1 is stationary, FLY in Sample2 is 1-order stationary, PCA FLY in Sample2 is stationary.

### 3. Perform Cointegration analysis either by Box-Tiao in Sample1. Alternatively, compute a COINT FLY from a level regression.

In [5]:
# Function to perform CCA via Box & Tiao decomposotion
def CCA_BT(df):
    # Calculate coefficient
    df_cov = df.cov()
    nsqrt = scipy.linalg.sqrtm(df_cov)
    x = df[:-1]
    y = df[1:]
    lr = LinearRegression(fit_intercept=False)
    v = lr.fit(x, y)
    A = v.coef_
    yp = np.dot(x, A)
    sqrt2 = np.dot(y.T, y)
     
    # decompose
    Q = inv(np.matrix(df_cov)) * np.matrix(A) * np.matrix(df_cov) * np.matrix(A.T)
    Eval, Evec = eigh(Q) 
    order = np.argsort(Eval)[0]
    Evec = Evec[:, order]
    return nsqrt, Evec

# Data to be decomposed
data_cca = Sample1_adjust[["2Y", "5Y", "10Y"]]
r = CCA_BT(data_cca)

# Compute the weights
w = np.dot(inv(r[0]), r[1])
w /= -w[1,0]
print('The (w1,w2) for butterfly porfolio weights computed by CCA are: \n',(w[0, 0], w[2, 0]))

# Level regression
l = LinearRegression()
x = Sample1[["2Y","10Y"]]
y = Sample1["5Y"]
l.fit(x, y)
w = l.coef_
print('\nThe (w1,w2) for butterfly porfolio weights computed by level regression are: \n', (w[0], w[1]))

('The (w1,w2) for butterfly porfolio weights computed by CCA are: \n', (2.1261763831944926, 0.62668761077133206))
('\nThe (w1,w2) for butterfly porfolio weights computed by level regression are: \n', (1.8343269691413886, 0.62525270878057082))


To decompose the data by CCA, instead of using package, I define the function by myself. First, adjust the original data to zero mean, then calculate the measure of relative predictibility, decompose it to get eigenvalue and eigenvector. Choose the eigenvector corresponding to the smallest eigenvalue of the matrix, multiplied by negative square root of covariance matrix, then we can get the weights and set the second weight to -1.

To perform level regression, just do linear regression on 2Y10Y and 5Y.

### 4. Compute Augmented Dickey-Fuller (ADF) test statistic for:
(1) COINT FLY in Sample1 , (2) COINT FLY in Sample2

In [6]:
# Prepare data to be tested
Sample1_COINT_FLY = np.dot(Sample1_2Y5Y10Y, [w[0], -1, w[1]])
Sample2_COINT_FLY = np.dot(Sample2_2Y5Y10Y, [w[0], -1, w[1]])

# Test
print('1. ADF test for COINT FLY in Sample1:')
ADF_test(Sample1_COINT_FLY)
print('\n\n2. ADF test for COINT FLY in Sample2:')
ADF_test(Sample2_COINT_FLY)

1. ADF test for COINT FLY in Sample1:
('adf: ', -4.8021863720609872)
('p-value: ', 5.3864588023024334e-05)
('critical values: ', {'5%': -2.8673495105661462, '1%': -3.4435228622952065, '10%': -2.569864247011056})
Time series is stationary.


2. ADF test for COINT FLY in Sample2:
('adf: ', -1.4958108616299031)
('p-value: ', 0.53558261851374178)
('critical values: ', {'5%': -2.8673378563200003, '1%': -3.4434963794639999, '10%': -2.5698580359999998})
('Time series is nonstationary.', '\n')
Start testing 1-order difference series...
('adf: ', -24.983095031542941)
('p-value: ', 0.0)
('critical values: ', {'5%': -2.8673612117611267, '1%': -3.4435494520411605, '10%': -2.5698704830567247})
1-order difference series is stationary.


From the ADF test, we can find that the COINT FLY is stationary in sample 1, but nonstationary in sample2. So we can find that the level regression is overfitting, which is not a good way to calculate the weights.

### Summary:

From 1 to 4, I performed PCA, CCA and ADF test. By computing the weights of portfolio, I find that w1 should be around 2.00, w2 should be around 0.62. According to ADF test, original yields time series are always nonstationary but the 1-order difference series , but weighted yields series are always stationary  